# Practice 3: Dataframes (solution)

## Initialization

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

##  Reading Datasets as DataFrames

In [ ]:
customers = sqlContext.read.format('com.databricks.spark.csv')\
                      .options(header='true', inferschema='true', delimiter='\t')\
                      .load('data/store/customers.tsv')

details = sqlContext.read.format('com.databricks.spark.csv')\
                    .options(header='true', inferschema='true', delimiter='\t')\
                    .load('data/store/order_details.tsv')

orders = sqlContext.read.format('com.databricks.spark.csv')\
                   .options(header='true', inferschema='true', delimiter='\t')\
                   .load('data/store/orders.tsv')

products = sqlContext.read.format('com.databricks.spark.csv')\
                     .options(header='true', inferschema='true', delimiter='\t')\
                     .load('data/store/products.tsv')

## API Solution

In [ ]:
sub_orders = orders.select('OrderID', 'CustomerID')
sub_clients = customers.select('CustomerID', 'Country')
sub_details = details.select('OrderID', 'ProductID', 'Quantity')
sub_products = products.select('ProductID', 'Price')

df = sub_orders.join(sub_clients, 'CustomerID')\
               .join(sub_details, 'OrderID')\
               .join(sub_products, 'ProductID')
        
df2 = df.withColumn('Total', df.Quantity * df.Price)\
        .select('OrderID', 'Country', 'Total')\
        .groupBy('OrderID', 'Country')\
        .agg({'Total' : 'sum'})

df3 = df2.groupBy('Country').agg({'sum(Total)' : 'avg'})

df3.orderBy('avg(sum(Total))', ascending=False).show(5)

## SQL Query Solution